##Other

### OS cd

In [1]:
import os
os.chdir("/content/drive/MyDrive/Ntu/DS/HW6/my_codes")
print(os.getcwd())

/content/drive/MyDrive/Ntu/DS/HW6/my_codes


###Module of main

In [2]:
import os
import pandas as pd

from utils.options import args
import utils.common as utils

from importlib import import_module

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR as MultiStepLR
from torch.optim.lr_scheduler import StepLR as StepLR

from data import dataPreparer

import warnings, math

Gobal of main

In [3]:
warnings.filterwarnings("ignore")

device = torch.device(f"cuda:{args.gpus[0]}")

checkpoint = utils.checkpoint(args)

## Main def

In [4]:
def main():

    start_epoch = 0
    best_acc = 0.0
 
    

    
    # Create model
    print('=> Building model...')

    # load training model
    model = import_module(f'model.{args.arch}').__dict__[args.model]().to(device)
    

    # Load pretrained weights
    if args.pretrained:
        print("=> Load pretrained weights") ##record
 
        ckpt = torch.load(args.source_dir + args.source_file, map_location = device)
        state_dict = ckpt['state_dict']

        model.load_state_dict(state_dict)
        model = model.to(device)
        
    if args.inference_only:
        acc = inference(args, data_loader_eval, model, args.output_file)
        print(f'Test acc {acc:.3f}\n')
        return

    param = [param for name, param in model.named_parameters()]
    
    optimizer = optim.SGD(param, lr = args.lr, momentum = args.momentum, weight_decay = args.weight_decay)
    scheduler = StepLR(optimizer, args.lr_decay_step, gamma = args.lr_gamma)


    for epoch in range(start_epoch, args.num_epochs):
        print(f"**epoch:{epoch+1}/{args.num_epochs}**", flush=True)  ## record
        scheduler.step(epoch)
        
        train(args, data_loader, model, optimizer, epoch)
        
        test_acc = test(args, data_loader_eval, model)
   
        is_best = best_acc < test_acc
        best_acc = max(best_acc, test_acc)
        

        state = {
            'state_dict': model.state_dict(),
            
            'optimizer': optimizer.state_dict(),
            
            'scheduler': scheduler.state_dict(),
            
            'epoch': epoch + 1
        }
        checkpoint.save_model(state, epoch + 1, is_best)
        
    inference(args, data_loader_eval, model, args.output_file)
    
    print(f'Best acc: {best_acc:.3f}\n')

## Train
per epoch

In [5]:
def train(args, data_loader, model, optimizer, epoch):
    losses = utils.AverageMeter()

    acc = utils.AverageMeter()

    criterion = nn.CrossEntropyLoss()
    
    num_iterations = len(data_loader)
    
    # switch to train mode
    model.train()

    
         
    for i, (inputs, targets, _) in enumerate(data_loader, 1):
        print(f"epoch:{epoch+1}/{args.num_epochs}, iter:{i}/{num_iterations}", flush=True)  ## record
        num_iters = num_iterations * epoch + i

        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        # train
        output = model(inputs)
        loss = criterion(output, targets)

        # optimize cnn
        loss.backward()
        optimizer.step()

        ## train weights        
        losses.update(loss.item(), inputs.size(0))
        
        ## evaluate
        prec1, _ = utils.accuracy(output, targets, topk = (1, 5))
        acc.update(prec1[0], inputs.size(0))

        
        if i % args.print_freq == 0:     
            print(
                'Epoch[{0}]({1}/{2}): \n'
                'Train_loss: {train_loss.val:.4f} ({train_loss.avg:.4f})\n'
                'Train acc {acc.val:.3f} ({acc.avg:.3f})\n'.format(
                epoch, i, num_iterations, 
                train_loss = losses,
                acc = acc))

## Test (Validation)

In [6]:
def test(args, loader_test, model):
    losses = utils.AverageMeter()
    acc = utils.AverageMeter()

    criterion = nn.CrossEntropyLoss()

    num_iterations_test = len(loader_test) ## record


    # switch to eval mode
    model.eval()

    with torch.no_grad():
        for i, (inputs, targets, datafile) in enumerate(loader_test, 1):
            print(f"Eval-- iter:{i}/{num_iterations_test}", flush=True) ## record
            
            inputs = inputs.to(device)
            targets = targets.to(device)
             
            preds = model(inputs)
            loss = criterion(preds, targets)
        
            # image classification results
            prec1, _ = utils.accuracy(preds, targets, topk = (1, 5))
            losses.update(loss.item(), inputs.size(0))
            acc.update(prec1[0], inputs.size(0))

 
    print(f'Test acc {acc.avg:.3f}\n')

    return acc.avg

## Inference

In [15]:
def inference(args, loader_test, model, output_file_name):
    losses = utils.AverageMeter()
    acc = utils.AverageMeter()

    criterion = nn.CrossEntropyLoss()
    num_iterations_test = len(loader_test) ## record

    outputs = []
    datafiles = []
    count = 1
    # switch to eval mode
    model.eval()

    with torch.no_grad():
        for i, (inputs, targets, datafile) in enumerate(loader_test, 1):
            print(f"Eval-- iter:{i}/{num_iterations_test}", flush=True) ## record
            
            inputs = inputs.to(device)
            targets = targets.to(device)
            
          
            preds = model(inputs)
    
            if args.src_label_path is not None:  ## new codes
              loss = criterion(preds, targets)
        
              # image classification results
              prec1, _ = utils.accuracy(preds, targets, topk = (1, 5))
              losses.update(loss.item(), inputs.size(0))
              acc.update(prec1[0], inputs.size(0))
            
            _, output = preds.topk(1, 1, True, True)
            
            outputs.extend(list(output.reshape(-1).cpu().detach().numpy()))
            
            datafiles.extend(list(datafile))
            
            count += inputs.size(0)
    

    output_file = dict()
    output_file['image_name'] = datafiles
    output_file['label'] = outputs
    
    output_file = pd.DataFrame.from_dict(output_file)

    def _make_dir(path):
            if not os.path.exists(path): os.makedirs(path)

    _make_dir(args.job_dir)
    output_file.to_csv(output_file_name, index = False)
    
    return acc.avg

## Control panel

control

In [16]:
args.__dict__

{'arch': 'cnn',
 'eval_batch_size': 32,
 'gpus': [0],
 'inference_only': True,
 'job_dir': 'experiment/train/',
 'lr': 0.01,
 'lr_decay_step': 20,
 'lr_gamma': 0.1,
 'model': 'CNN',
 'momentum': 0.9,
 'num_epochs': 10,
 'output_file': 'experiment/test/res.csv',
 'pretrained': True,
 'print_freq': 5,
 'refine': None,
 'reset': False,
 'resume': None,
 'source_dir': 'pretrained/',
 'source_file': 'model_best.pt',
 'src_data_path': '../digit/test',
 'src_label_path': None,
 'test_only': False,
 'train_batch_size': 64,
 'weight_decay': 0.0005}

In [17]:
args.output_file.split('/')

['experiment', 'test', 'res.csv']

In [18]:
working_mode = 'test'


if working_mode == 'test':
  args.src_data_path = '../digit/test'
  args.src_label_path = None
  args.job_dir = 'experiment/test/'
  args.output_file = 'experiment/test/res.csv'
  args.pretrained = True
  args.inference_only = True

elif working_mode == 'valid':
  args.src_data_path = '../digit/train'
  args.src_label_path = '../digit/train.csv'
  args.job_dir = 'experiment/train/'
  args.output_file = 'experiment/train/output.csv'
  args.pretrained = True
  args.inference_only = True

elif working_mode == 'train':
  args.src_data_path = '../digit/train'
  args.src_label_path = '../digit/train.csv'
  args.job_dir = 'experiment/train/'
  args.output_file = 'experiment/train/output.csv'
  args.pretrained = False
  args.inference_only = False
else:
  raise BaseException("No matching working mode")



In [19]:
args.__dict__

{'arch': 'cnn',
 'eval_batch_size': 32,
 'gpus': [0],
 'inference_only': True,
 'job_dir': 'experiment/test/',
 'lr': 0.01,
 'lr_decay_step': 20,
 'lr_gamma': 0.1,
 'model': 'CNN',
 'momentum': 0.9,
 'num_epochs': 10,
 'output_file': 'experiment/test/res.csv',
 'pretrained': True,
 'print_freq': 5,
 'refine': None,
 'reset': False,
 'resume': None,
 'source_dir': 'pretrained/',
 'source_file': 'model_best.pt',
 'src_data_path': '../digit/test',
 'src_label_path': None,
 'test_only': False,
 'train_batch_size': 64,
 'weight_decay': 0.0005}

### Data loading...
NOTE: Do not run when you don't change the dir

In [11]:
# Data loading
print('=> Preparing data..')

# data loader

loader = dataPreparer.Data(args,data_path=args.src_data_path,label_path=args.src_label_path)

data_loader = loader.loader_train
data_loader_eval = loader.loader_test

=> Preparing data..


##main

In [20]:
if __name__ == '__main__':
    main()

=> Building model...
=> Load pretrained weights
Eval-- iter:1/157
Eval-- iter:2/157
Eval-- iter:3/157
Eval-- iter:4/157
Eval-- iter:5/157
Eval-- iter:6/157
Eval-- iter:7/157
Eval-- iter:8/157
Eval-- iter:9/157
Eval-- iter:10/157
Eval-- iter:11/157
Eval-- iter:12/157
Eval-- iter:13/157
Eval-- iter:14/157
Eval-- iter:15/157
Eval-- iter:16/157
Eval-- iter:17/157
Eval-- iter:18/157
Eval-- iter:19/157
Eval-- iter:20/157
Eval-- iter:21/157
Eval-- iter:22/157
Eval-- iter:23/157
Eval-- iter:24/157
Eval-- iter:25/157
Eval-- iter:26/157
Eval-- iter:27/157
Eval-- iter:28/157
Eval-- iter:29/157
Eval-- iter:30/157
Eval-- iter:31/157
Eval-- iter:32/157
Eval-- iter:33/157
Eval-- iter:34/157
Eval-- iter:35/157
Eval-- iter:36/157
Eval-- iter:37/157
Eval-- iter:38/157
Eval-- iter:39/157
Eval-- iter:40/157
Eval-- iter:41/157
Eval-- iter:42/157
Eval-- iter:43/157
Eval-- iter:44/157
Eval-- iter:45/157
Eval-- iter:46/157
Eval-- iter:47/157
Eval-- iter:48/157
Eval-- iter:49/157
Eval-- iter:50/157
Eval-- iter